# Data Loadin

## Sentiment Data

### TweetEval

In [1]:
import os
import pandas as pd

/Users/penglishao/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [1]:
from datetime import timedelta
N_sent, N_sarc = 19494, 6235
N = N_sent + N_sarc
P = 3.09   # 换成你最新测到的吞吐（CPU 或 MPS）
eta_seconds = N / P
print("ETA:", timedelta(seconds=int(eta_seconds)))

ETA: 2:18:46


In [2]:
ROOT = "/Users/penglishao/Desktop/DS5500/project"   # 如果在别的路径运行，改成你的 project 根，例如 "/Users/xxx/project"
TWEETEVAL_DIR = os.path.join(ROOT, "data", "tweetEval")

SENTIMENT_DIR = os.path.join(TWEETEVAL_DIR, "sentiment")
IRONY_DIR     = os.path.join(TWEETEVAL_DIR, "irony")

# 输出目录
OUT_DIR = os.path.join(ROOT, "data", "processed")
os.makedirs(OUT_DIR, exist_ok=True)

In [3]:
# ====== 2) 读取一个子任务（sentiment 或 irony）某个 split 的文本和标签 ======
def load_split(task_dir, split):
    """
    task_dir: e.g., data/tweetEval/sentiment  或  data/tweetEval/irony
    split: "train" | "val" | "test"
    returns: pd.DataFrame with columns ["text", "label_id"]
    """
    text_fp  = os.path.join(task_dir, f"{split}_text.txt")
    label_fp = os.path.join(task_dir, f"{split}_labels.txt")
    # 读文本
    with open(text_fp, "r", encoding="utf-8") as f:
        texts = [line.rstrip("\n") for line in f]
    # 读标签
    with open(label_fp, "r", encoding="utf-8") as f:
        labels = [int(line.strip()) for line in f]

    if len(texts) != len(labels):
        raise ValueError(f"Text/label length mismatch in {task_dir}, {split}: {len(texts)} vs {len(labels)}")
    return pd.DataFrame({"text": texts, "label_id": labels})

In [4]:
# ====== 3) 组装 sentiment_df ======
# 你的需求：0=neg, 1=neu, 2=pos
sentiment_label_map_id2name = {0: "negative", 1: "neutral", 2: "positive"}

dfs_sentiment = []
for split in ["train", "val", "test"]:
    df = load_split(SENTIMENT_DIR, split)
    # 保留原始 id，同时映射到我们需要的列名 "sentiment"
    df["sentiment"] = df["label_id"].map({0:0, 1:1, 2:2})  # 显式映射，确保一致
    df["sentiment_name"] = df["label_id"].map(sentiment_label_map_id2name)
    df["split"] = split
    dfs_sentiment.append(df[["text", "sentiment", "sentiment_name", "split"]])

sentiment_df = pd.concat(dfs_sentiment, ignore_index=True)

In [8]:
sentiment_df.head()

,text,sentiment,sentiment_name,split
0,"""QT @user In the original draft of the 7th boo...",2,positive,train
1,"""Ben Smith / Smith (concussion) remains out of...",1,neutral,train
2,Sorry bout the stream last night I crashed out...,1,neutral,train
3,Chase Headley's RBI double in the 8th inning o...,1,neutral,train
4,@user Alciato: Bee will invest 150 million in ...,2,positive,train


In [9]:
# ====== 4) 组装 sarcasm_df（irony 子任务） ======
# 你的需求：0=non-sarcastic, 1=sarcastic
sarcasm_label_map_id2name = {0: "non-sarcastic", 1: "sarcastic"}

dfs_sarcasm = []
for split in ["train", "val", "test"]:
    df = load_split(IRONY_DIR, split)
    df["sarcasm"] = df["label_id"].map({0:0, 1:1})
    df["sarcasm_name"] = df["label_id"].map(sarcasm_label_map_id2name)
    df["split"] = split
    dfs_sarcasm.append(df[["text", "sarcasm", "sarcasm_name", "split"]])

sarcasm_df = pd.concat(dfs_sarcasm, ignore_index=True)

In [10]:
sarcasm_df.head()

,text,sarcasm,sarcasm_name,split
0,seeing ppl walking w/ crutches makes me really...,1,sarcastic,train
1,"look for the girl with the broken smile, ask h...",0,non-sarcastic,train
2,Now I remember why I buy books online @user #s...,1,sarcastic,train
3,@user @user So is he banded from wearing the c...,1,sarcastic,train
4,Just found out there are Etch A Sketch apps. ...,1,sarcastic,train


In [11]:
# ====== 5) 简要检查 ======
print("Sentiment splits:\n", sentiment_df["split"].value_counts(), "\n")
print("Sentiment label counts:\n", sentiment_df["sentiment_name"].value_counts(), "\n")

print("Sarcasm splits:\n", sarcasm_df["split"].value_counts(), "\n")
print("Sarcasm label counts:\n", sarcasm_df["sarcasm_name"].value_counts(), "\n")

print("Examples (sentiment):")
print(sentiment_df.sample(3, random_state=42), "\n")
print("Examples (sarcasm):")
print(sarcasm_df.sample(3, random_state=42), "\n")


Sentiment splits:
 split
train    45615
test     12284
val       2000
Name: count, dtype: int64 

Sentiment label counts:
 sentiment_name
neutral     27479
positive    21043
negative    11377
Name: count, dtype: int64 

Sarcasm splits:
 split
train    2862
val       955
test      784
Name: count, dtype: int64 

Sarcasm label counts:
 sarcasm_name
non-sarcastic    2389
sarcastic        2212
Name: count, dtype: int64 

Examples (sentiment):
                                                    text  sentiment  \
45192  @user @user BJP may be liking Rahul Gandhi now...          2   
22309  The guy who let me borrow his Ohio State bag F...          2   
33992  @user @user will NEVER but the sun Jo\u002c ne...          0   

      sentiment_name  split  
45192       positive  train  
22309       positive  train  
33992       negative  train   

Examples (sarcasm):
                                                   text  sarcasm  \
3683  In Scotland ( the little country that's attach...       

In [12]:
# ====== 6) 保存文件 ======
sentiment_csv  = os.path.join(OUT_DIR, "sentiment_df.csv")
sarcasm_csv    = os.path.join(OUT_DIR, "sarcasm_df.csv")
sentiment_parq = os.path.join(OUT_DIR, "sentiment_df.parquet")
sarcasm_parq   = os.path.join(OUT_DIR, "sarcasm_df.parquet")

sentiment_df.to_csv(sentiment_csv, index=False)
sarcasm_df.to_csv(sarcasm_csv, index=False)
try:
    sentiment_df.to_parquet(sentiment_parq, index=False)
    sarcasm_df.to_parquet(sarcasm_parq, index=False)
except Exception as e:
    print("Parquet save failed (ok to ignore if pyarrow not installed):", e)

print("Saved to:\n", sentiment_csv, "\n", sarcasm_csv)

Saved to:
 /Users/penglishao/Desktop/DS5500/project/data/processed/sentiment_df.csv 
 /Users/penglishao/Desktop/DS5500/project/data/processed/sarcasm_df.csv


### Stanford Sentiment Treebank

In [ ]:
from huggingface_hub import login

login(token="YourToken")

from datasets import load_dataset

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/penglishao/.cache/huggingface/token
Login successful


In [22]:
sst = load_dataset("SetFit/sst5")

Found cached dataset json (/Users/penglishao/.cache/huggingface/datasets/SetFit___json/SetFit--sst5-4c07b9d5881ae209/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
sst

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
})

In [24]:
# 2) 读取标签名字（如果是 ClassLabel）
label_names = None
if "label" in sst["train"].features and hasattr(sst["train"].features["label"], "names"):
    label_names = sst["train"].features["label"].names  # e.g. ['very negative','negative','neutral','positive','very positive']


In [28]:
# 3) 5类 -> 3类的映射
map_5to3_name = {
    "very negative": "negative",
    "negative": "negative",
    "neutral": "neutral",
    "positive": "positive",
    "very positive": "positive",
}
sentiment_id_map = {"negative": 0, "neutral": 1, "positive": 2}

def split_name_for_te(s):  # 统一 split 命名：train/val/test
    return {"train":"train", "validation":"val", "test":"test"}[s]

def to_df(split):
    # to pandas
    df = sst[split].to_pandas()

    # 自动识别文本列名
    text_col = "text" if "text" in df.columns else ("sentence" if "sentence" in df.columns else None)
    if text_col is None:
        raise ValueError(f"无法在 SetFit/sst5 的 {split} split 里找到文本列（期望 'text' 或 'sentence'）")

    # 取 label 原始名字
    if "label_text" in df.columns:
        label_5 = df["label_text"].astype(str)
    elif "label" in df.columns:
        if label_names is None:
            # 兜底：假设是 0..4 的整数，并按常见顺序映射
            fallback = {0:"very negative", 1:"negative", 2:"neutral", 3:"positive", 4:"very positive"}
            label_5 = df["label"].map(fallback).astype(str)
        else:
            label_5 = df["label"].map(lambda i: label_names[int(i)]).astype(str)
    else:
        raise ValueError(f"未找到 label 列（期望 'label' 或 'label_text'） in {split}")

    # 5类 -> 3类（名字）
    label_3_name = label_5.map(map_5to3_name)

    # 名字 -> id（0/1/2）
    label_3_id = label_3_name.map(sentiment_id_map).astype(int)

    # 组织为统一结构
    out = pd.DataFrame({
        "text": df[text_col].astype(str),
        "sentiment_name": label_3_name,
        "sentiment": label_3_id,
        "split": split_name_for_te(split),
    })

    # 基本清洗：去空白
    out["text"] = out["text"].str.strip()
    return out[["text", "sentiment", "sentiment_name", "split"]]


In [ ]:
# 4) 生成 sst5 的三分类 DataFrame
sst5_sentiment_df = pd.concat(
    [to_df("train"), to_df("validation"), to_df("test")],
    ignore_index=True
)

print(sst5_sentiment_df.head())
print(sst5_sentiment_df["split"].value_counts())
print(sst5_sentiment_df["sentiment_name"].value_counts())

                                                text  sentiment  \
0  a stirring , funny and finally transporting re...          2   
1  apparently reassembled from the cutting-room f...          0   
2  they presume their audience wo n't sit still f...          0   
3  the entire movie is filled with deja vu moments .          1   
4  this is a visually stunning rumination on love...          2   

  sentiment_name  split  
0       positive  train  
1       negative  train  
2       negative  train  
3        neutral  train  
4       positive  train  
split
train    8544
test     2210
val      1101
Name: count, dtype: int64
sentiment_name
positive    4963
negative    4650
neutral     2242
Name: count, dtype: int64


In [34]:
sst5_sentiment_df.head()

,text,sentiment,sentiment_name,split
0,"a stirring , funny and finally transporting re...",2,positive,train
1,apparently reassembled from the cutting-room f...,0,negative,train
2,they presume their audience wo n't sit still f...,0,negative,train
3,the entire movie is filled with deja vu moments .,1,neutral,train
4,this is a visually stunning rumination on love...,2,positive,train


In [36]:
# 6) （可选）保存
sst5_sentiment_df.to_csv("../data/processed/sentiment_sst5_mapped.csv", index=False)


### MARC

In [43]:
MARC_ds = load_dataset("mteb/amazon_reviews_multi", "en")

Found cached dataset amazon_reviews_multi (/Users/penglishao/.cache/huggingface/datasets/mteb___amazon_reviews_multi/en/1.0.0/61154b14772a2d73f3554caa8f5bd1fec4b65ec64f70bb93fa9fec6b038a4355)


  0%|          | 0/3 [00:00<?, ?it/s]

In [44]:

# 2) 1–5 星 -> 三分类 (0=neg,1=neu,2=pos)
def stars_to_sentiment_id(stars: int) -> int:
    if stars in (1, 2):
        return 0
    elif stars == 3:
        return 1
    elif stars in (4, 5):
        return 2
    else:
        raise ValueError(f"Unexpected stars value: {stars}")

sentiment_id_to_name = {0: "negative", 1: "neutral", 2: "positive"}

def norm_split(s): 
    return {"train": "train", "validation": "val", "test": "test"}[s]

In [45]:
# 3) 从 label / label_text 推断星级（1..5）
def infer_stars(df: pd.DataFrame) -> pd.Series:
    if "label" in df.columns:
        lab = pd.to_numeric(df["label"], errors="coerce")
        # 如果是 0..4 索引 → +1 变成 1..5
        if lab.dropna().between(0, 4).all():
            return (lab + 1).astype(int)
        # 如果已经是 1..5，原样返回
        if lab.dropna().between(1, 5).all():
            return lab.astype(int)
    if "label_text" in df.columns:
        # 例如 "1 star" / "5 stars" / "3"
        return df["label_text"].astype(str).str.extract(r'(\d)').astype(int)[0]
    raise ValueError(f"Cannot infer stars; columns present: {list(df.columns)}")


In [46]:

# 4) 取文本列
def get_text_col(df: pd.DataFrame) -> pd.Series:
    if "text" in df.columns:
        return df["text"].astype(str)
    # 兜底：找第一个字符串列
    text_cols = df.select_dtypes(include=["object"]).columns.tolist()
    if text_cols:
        return df[text_cols[0]].astype(str)
    raise ValueError("No text column found (expected 'text').")

def to_df(split):
    hf = MARC_ds[split].to_pandas()
    text = get_text_col(hf).str.strip()
    stars = infer_stars(hf)
    sent_id = stars.map(stars_to_sentiment_id).astype(int)
    sent_name = sent_id.map(sentiment_id_to_name)

    out = pd.DataFrame({
        "text": text,
        "sentiment": sent_id,              # 0=neg,1=neu,2=pos
        "sentiment_name": sent_name,
        "split": norm_split(split),
    })
    return out[["text", "sentiment", "sentiment_name", "split"]]


In [47]:

# 5) 生成 DataFrame（MARC -> 三分类 sentiment）
marc_sentiment_df = pd.concat(
    [to_df("train"), to_df("validation"), to_df("test")],
    ignore_index=True
)

# 6) 简要检查
print(marc_sentiment_df["split"].value_counts())
print(marc_sentiment_df["sentiment_name"].value_counts())
print(marc_sentiment_df.sample(3, random_state=0))

split
train    200000
val        5000
test       5000
Name: count, dtype: int64
sentiment_name
negative    84000
positive    84000
neutral     42000
Name: count, dtype: int64
                                                     text  sentiment  \
131072    It is functional.\n\nVery good. I recommend it.          2   
61816   Hurt my feet\n\nI don’t know why but I just di...          0   
28020   Pic not what was sent\n\nWhat they show in the...          0   

       sentiment_name  split  
131072       positive  train  
61816        negative  train  
28020        negative  train  


In [52]:
marc_sentiment_df.head()

,text,sentiment,sentiment_name,split
0,I'll spend twice the amount of time boxing up ...,0,negative,train
1,Not use able\n\nthe cabinet dot were all detac...,0,negative,train
2,The product is junk.\n\nI received my first or...,0,negative,train
3,Fucking waste of money\n\nThis product is a pi...,0,negative,train
4,bubble\n\nwent through 3 in one day doesn't fi...,0,negative,train


In [53]:
marc_sentiment_df.to_csv("../data/processed/sentiment_marc_mapped.csv", index=False)


## Sarcasm Data

### SARC

### SemEval

In [57]:
BASE_DIR = "/Users/penglishao/Desktop/DS5500/project"  # 改成你的项目根目录
IN_DIR = os.path.join(BASE_DIR, "data", "SemEval")
OUT_DIR = os.path.join(BASE_DIR, "data", "processed", "sarcasm_data")
os.makedirs(OUT_DIR, exist_ok=True)


FILES = [
    "SemEval2018-T3-train-taskA.txt",
    "SemEval2018-T3-train-taskB.txt",
]

EXPECTED_COLS = ["Tweet Index", "Label", "Tweet text"]


In [58]:
def read_semeval_file(path):
    """
    读取单个 SemEval-2018 Task 3 文件（TSV）。
    自动处理是否有表头。如果没有表头，赋予 EXPECTED_COLS。
    """
    # 先读前几行判断是否有表头
    with open(path, "r", encoding="utf-8") as f:
        head = [next(f).rstrip("\n") for _ in range(3)]
    header_line = head[0]
    has_header = all(col in header_line for col in ["Tweet", "Label"])  # 粗略判断

    if has_header:
        df = pd.read_csv(path, sep="\t", encoding="utf-8")
        # 兼容不同大小写/空格写法
        colmap = {c: c.strip() for c in df.columns}
        df.rename(columns=colmap, inplace=True)
        # 尝试对齐列名
        rename_guess = {}
        for c in df.columns:
            lc = c.lower().strip()
            if "tweet" in lc and "text" in lc:
                rename_guess[c] = "Tweet text"
            elif lc.startswith("tweet") and "index" in lc:
                rename_guess[c] = "Tweet Index"
            elif lc == "label":
                rename_guess[c] = "Label"
        if rename_guess:
            df.rename(columns=rename_guess, inplace=True)
    else:
        # 无表头：直接按三列读
        df = pd.read_csv(path, sep="\t", header=None, names=EXPECTED_COLS, encoding="utf-8")

    # 只保留需要的三列（防止多余列干扰）
    for need in EXPECTED_COLS:
        if need not in df.columns:
            raise ValueError(f"Missing expected column '{need}' in file: {path} (cols={list(df.columns)})")
    df = df[EXPECTED_COLS].copy()
    return df

In [59]:

# 1) 读取并合并 taskA / taskB
dfs = []
for fn in FILES:
    fp = os.path.join(IN_DIR, fn)
    if not os.path.exists(fp):
        raise FileNotFoundError(f"Not found: {fp}")
    df_part = read_semeval_file(fp)
    dfs.append(df_part)

raw_df = pd.concat(dfs, ignore_index=True)

In [60]:
# 2) 规范化：去空白、丢弃空文本
raw_df["Tweet text"] = raw_df["Tweet text"].astype(str).str.strip()
raw_df.dropna(subset=["Tweet text", "Label"], inplace=True)
raw_df = raw_df[raw_df["Tweet text"] != ""]

# 3) 标签映射：0 -> non-sarcastic；其他 -> sarcastic
def label_to_sarcasm(x):
    try:
        xi = int(x)
    except Exception:
        # 如果是字符串，尽力抽取数字
        try:
            xi = int(str(x).strip().split()[0])
        except Exception:
            raise ValueError(f"Unrecognized label value: {x}")
    return 0 if xi == 0 else 1

sarcasm = raw_df["Label"].map(label_to_sarcasm).astype(int)
sarcasm_name = sarcasm.map({0: "non-sarcastic", 1: "sarcastic"})

In [61]:
# 4) 生成目标 DataFrame
out_df = pd.DataFrame({
    "text": raw_df["Tweet text"],
    "sarcasm": sarcasm,
    "sarcasm_name": sarcasm_name,
    "split": "train",
})

In [62]:
# 5) 去重（可选）
out_df.drop_duplicates(subset=["text", "sarcasm"], inplace=True)

# 6) 简要检查
print("Split counts:", out_df["split"].value_counts().to_dict())
print("Label counts:", out_df["sarcasm_name"].value_counts().to_dict())
print(out_df.sample(5, random_state=42))

Split counts: {'train': 3821}
Label counts: {'non-sarcastic': 1920, 'sarcastic': 1901}
                                                   text  sarcasm  \
2755  Security Engineers are not Security Guards. :|...        1   
1425  Taking a final then going straight to work. I'...        1   
2186  @Katerintree obvi because every Jesuit is a he...        1   
3497    This history essay is literally shitting on me.        0   
3049  I refuse to be weak... #workout #motivation #f...        0   

       sarcasm_name  split  
2755      sarcastic  train  
1425      sarcastic  train  
2186      sarcastic  train  
3497  non-sarcastic  train  
3049  non-sarcastic  train  


In [63]:
out_df.head()

,text,sarcasm,sarcasm_name,split
0,Sweet United Nations video. Just in time for C...,1,sarcastic,train
1,@mrdahl87 We are rumored to have talked to Erv...,1,sarcastic,train
2,Hey there! Nice to see you Minnesota/ND Winter...,1,sarcastic,train
3,3 episodes left I'm dying over here,0,non-sarcastic,train
4,I can't breathe! was chosen as the most notabl...,1,sarcastic,train


In [64]:
# 7) 保存
csv_path = os.path.join(OUT_DIR, "semeval2018_irony_train.csv")
out_df.to_csv(csv_path, index=False, encoding="utf-8")

print("Saved:", csv_path)

Saved: /Users/penglishao/Desktop/DS5500/project/data/processed/sarcasm_data/semeval2018_irony_train.csv


### iSarcasmEval

In [65]:
BASE = "/Users/penglishao/Desktop/DS5500/project"  # 改成你的项目根目录
IN_DIR = os.path.join(BASE, "data", "iSarcasmEval")
OUT_DIR = os.path.join(BASE, "data", "processed", "sarcasm_data")
os.makedirs(OUT_DIR, exist_ok=True)

In [66]:
def clean_text(s):
    return (str(s) if pd.notna(s) else "").strip()

def make_df(texts, labels, split):
    df = pd.DataFrame({
        "text": [clean_text(t) for t in texts],
        "sarcasm": [int(x) for x in labels],
        "split": split
    })
    df = df[df["text"] != ""].copy()
    df["sarcasm_name"] = df["sarcasm"].map({0: "non-sarcastic", 1: "sarcastic"})
    return df[["text", "sarcasm", "sarcasm_name", "split"]]

all_parts = []

In [68]:
# ========= 1) Train =========
train_fp = os.path.join(IN_DIR, "train.En.csv")
train = pd.read_csv(train_fp)

# 期望字段: tweet, sarcastic, rephrase, 以及类别列（英）:
# sarcasm, irony, satire, understatement, overstatement, rhetorical_question
# a) 原tweet（带标签）
train_tweets = make_df(train["tweet"], train["sarcastic"], split="train")

# b) rephrase -> 全部当作 non-sarcastic=0，单独作为新样本
if "rephrase" in train.columns:
    rephr = train["rephrase"].fillna("").astype(str)
    # 只保留非空重述
    mask = rephr.str.strip() != ""
    train_rephr = make_df(rephr[mask], [0]*mask.sum(), split="train")
    all_parts += [train_tweets, train_rephr]
else:
    all_parts += [train_tweets]

In [69]:

# ========= 2) Task A (test) =========
# columns: text, sarcastic (0/1)
taskA_fp = os.path.join(IN_DIR, "task_A_En_test.csv")
if os.path.exists(taskA_fp):
    taskA = pd.read_csv(taskA_fp)
    # 有的版本列名可能是 'sarcastic' 或 'label'
    labA = taskA["sarcastic"] if "sarcastic" in taskA.columns else taskA["label"]
    partA = make_df(taskA["text"], labA, split="test")
    all_parts.append(partA)

In [70]:
# ========= 3) Task B (test) =========
# columns example: text, sarcasm, irony, satire, understatement, overstatement, rhetorical_question
# 若无 'sarcasm' 列，则可由多标签是否有任意一类=1 来近似推断讽刺（保险起见，这里优先用 'sarcasm' 列）
taskB_fp = os.path.join(IN_DIR, "task_B_En_test.csv")
if os.path.exists(taskB_fp):
    taskB = pd.read_csv(taskB_fp)
    if "sarcasm" in taskB.columns:
        labB = taskB["sarcasm"].astype(int)
    else:
        cat_cols = [c for c in ["irony","satire","understatement","overstatement","rhetorical_question"] if c in taskB.columns]
        if not cat_cols:
            raise ValueError("Task B file found but cannot infer sarcasm (no 'sarcasm' or category columns).")
        labB = (taskB[cat_cols].sum(axis=1) > 0).astype(int)
    partB = make_df(taskB["text"], labB, split="test")
    all_parts.append(partB)

In [71]:

# ========= 4) Task C (test) =========
# columns: text_0, text_1, sarcastic_id (0/1)
# 需要把每行拆为两条样本：text_0 的讽刺标记 = (sarcastic_id==0)；text_1 的讽刺标记 = (sarcastic_id==1)
taskC_fp = os.path.join(IN_DIR, "task_C_En_test.csv")
if os.path.exists(taskC_fp):
    taskC = pd.read_csv(taskC_fp)
    # a) text_0
    labC0 = (taskC["sarcastic_id"].astype(int) == 0).astype(int)
    partC0 = make_df(taskC["text_0"], labC0, split="test")
    # b) text_1
    labC1 = (taskC["sarcastic_id"].astype(int) == 1).astype(int)
    partC1 = make_df(taskC["text_1"], labC1, split="test")
    all_parts += [partC0, partC1]

In [72]:

# ========= 5) 合并、去重 =========
sarcasm_df = pd.concat(all_parts, ignore_index=True)
# 可按文本+标签去重（避免重复样本）
sarcasm_df.drop_duplicates(subset=["text", "sarcasm"], inplace=True)

In [73]:
# ========= 6) 简要检查 =========
print("Splits:", sarcasm_df["split"].value_counts().to_dict())
print("Labels:", sarcasm_df["sarcasm_name"].value_counts().to_dict())
print(sarcasm_df.sample(5, random_state=1))

Splits: {'train': 4326, 'test': 1630}
Labels: {'non-sarcastic': 4781, 'sarcastic': 1175}
                                                   text  sarcasm  \
3589  I just fell down the stairs after holding the ...        0   
5251  Woke up from a weird stress dream about our of...        0   
4117  "Screw the kid who brought a pineapple to home...        0   
2310  Sorry I've not been that active in chat! I hav...        0   
7484  Great start to a monday! Find your patience be...        1   

       sarcasm_name  split  
3589  non-sarcastic  train  
5251  non-sarcastic   test  
4117  non-sarcastic  train  
2310  non-sarcastic  train  
7484      sarcastic   test  


In [74]:
# ========= 7) 保存 =========
out_csv = os.path.join(OUT_DIR, "isarcasmeval_merged.csv")
sarcasm_df.to_csv(out_csv, index=False, encoding="utf-8")

print("Saved:", out_csv)

Saved: /Users/penglishao/Desktop/DS5500/project/data/processed/sarcasm_data/isarcasmeval_merged.csv
